In [1]:
import pandas as pd
from glob import glob

In [ ]:
dfs = []
for fname in glob("/Users/francois.weber/Downloads/xa*"):

    df = pd.read_json(fname, lines=True)
    
    # only keep FR
    df = df.query("lang_code == 'fr'")
    
    # exclude categories 
    exclude_pos_titles = [
        "Locution nominale", # ex: barbe à papa, bande dessinée 
        "Nom propre",
        "Locution interjective", # ex: joyeux Noël 
        "Locution-phrase", # ex: je t'aime
        "Locution prépositive", 
        "Locution verbale", # exemple : en avoir plein le dos 
        "Locution adjectivale", # exemple : en avoir plein le dos 
        "Locution conjonctive", # ex: vu que
        "Forme de locution nominale", 
        "Proverbe",
        "Particule", 
        "Variante par contrainte typographique", # ex: coeur de métier
        "Pronom possessif", # ex: le mien
        "Forme de locution adjectivale", # ex: cousu de fil blanc
        "Adjectif numéral", # un, deux 
        "Forme de locution conjonctive", # ex: bien qu'
        "Prénom",
        "Nom de famille",
        "Forme de locution verbale", #ex: cru bon
        "Symbole",
        "Suffixe",
        "Forme de variante par contrainte typographique"
        "Préfixe"
    ]

    for exclude_pos_title in exclude_pos_titles:
        df = df[~df.pos_title.str.startswith(exclude_pos_title)]

    # categories that should be flagged as optional 
    not_really_words_pos_titles = [
        "Locution adverbiale",
        "Locution conjonctive",
        "Locution pronominale",
        "Pronom possessif",
        "Pronom indéfini",
        "Variante par contrainte typographique",
        "Forme de locution nominale",
        "Adjectif indéfini",
        "Particule",
        "Locution adverbiale 1",
        "Locution adverbiale 2",
        "Locution adverbiale 3",
        "Onomatopée",
        "Postposition",
    ]

    df["optional_category"] = df.pos_title.isin(not_really_words_pos_titles)
    
    # Exclude derivatives of verbs 
    def is_conjugated_verb(word: pd.Series) -> bool:
        return word.pos_title == "Forme de verbe"

    df["is_conjugated_verb"] = df.apply(is_conjugated_verb, axis=1)
    df = df[~df.is_conjugated_verb]
    
    # deal with derived forms of other words
    def is_form_of_other_word(word: pd.Series) -> bool:
        for sense in word.senses:
            if (forms_of:=sense.get("form_of", None)) is not None:
                for form_of in forms_of:
                    if "word" in form_of:
                        return True 
        return False

    df["is_form_of_other_word"] = df.apply(is_form_of_other_word, axis=1)
    df = df[~df.is_form_of_other_word]
    
    # Deal with specialmulti-word expressions 
    def word_has_spaces(word: pd.Series) -> bool:
        return " " in word.word

    df["word_has_spaces"] = df.apply(word_has_spaces, axis=1)

    dissallow_with_spaces_pos_titles = [
        "Adjectif indéfini", # ex: n'importe quel
        "Pronom indéfini", # ex: les deux
        "Pronom interrogatif", # ex: ce qui
        "Pronom relatif", # ex: ce qui
        "Adverbe interrogatif", # ex: comment ça
        "Forme d’article partitif", # ex: de la
    ]

    for pos_title in dissallow_with_spaces_pos_titles:
        df = df[~(df.word_has_spaces & (df.pos_title == pos_title))]
        
    # deal with gentillés
    def is_gentile(word: pd.Series) -> bool:
        descr = str(word.categories) + str(word.senses)
        return "gentilé" in descr.lower()

    df["is_gentile"] = df.apply(is_gentile, axis=1)
    df = df[~df.is_gentile]
    
    # remove acronyms 
    df = df[~(df.word.str.upper() == df.word)]
    
    # remove digits based words
    def contains_digit(word: str):
        return any([str(digit) in word for digit in range(10)])
    
    df = df[~df.word.apply(contains_digit)]
    
    # move on
    print(len(dfs) + 1, "ok")
    dfs.append(df.assign(from_chunk=i))

KeyboardInterrupt: 

In [57]:
df = pd.concat(dfs).reset_index(drop=True)[["word", "pos", "pos_title", "senses", "tags", "optional_category", "from_chunk"]]

# re process smaller dataset

In [60]:
from typing import List


def extract_definitions(senses: pd.Series) -> List[str]:
    defs = []
    for sense in senses:
        if "glosses" in sense:
            defs.extend(sense["glosses"])
    return defs


In [61]:
df["definitions"] = df.pop("senses").apply(extract_definitions)

In [62]:
# No chemistry
df = df[~df.definitions.apply(lambda ll: "".join(ll)).str.lower().str.contains("chimi")]

In [64]:
df.sample(10)

,word,pos,pos_title,tags,optional_category,definitions
124750,ménétrière,noun,Nom commun,[feminine],False,[Violoniste de village qui escorte notamment l...
5832,papeda,noun,Nom commun,[masculine],False,"[Agrume du sous-genre Papeda du genre Citrus, ..."
124752,ménétrolois,adj,Adjectif,[masculine],False,"[Relatif à Ménétrol, commune française située ..."
189888,blandonnée,noun,Nom commun,[feminine],False,[Méteil.]
149096,strapping,noun,Nom commun,[masculine],False,[Bandage compressif utilisé pour immobiliser u...
9552,sans cérémonie,adv,Locution adverbiale,[invariable],True,"[Sans façons, de manière informelle.]"
147313,redégraisser,verb,Verbe,[transitive],False,[Dégraisser à nouveau.]
225157,euphratien,adj,Adjectif,NaN,False,[Relatif à l'Euphrate ou à sa vallée.]
7494,directeur-gérant,noun,Nom commun,[masculine],False,[Directeur qui exerce aussi les fonctions de g...
55810,rinçure,noun,Nom commun,[feminine],False,"[Eau avec laquelle on a rincé un verre, une bo..."


In [71]:
df.to_parquet("/Users/francois.weber/code/french-words-covering/dico-fr.parquet")

In [73]:
a = pd.read_json(fname, lines=True)

In [74]:
a

,word,lang_code,lang,pos,pos_title,senses,categories,tags,raw_tags,etymology_texts,...,hypernyms,title,redirect,hyponyms,etymology_examples,paronyms,holonyms,abbreviation,meronyms,proverbs
0,оперте,uk,Ukrainien,adj,Forme d’adjectif,[{'glosses': ['Nominatif neutre singulier de о...,"[Formes d’adjectifs en ukrainien, ukrainien]",[form-of],[⁽ᵒᵖᵉʳᵗᵉ⁾],NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,опертим,uk,Ukrainien,adj,Forme d’adjectif,[{'glosses': ['Instrumental masculin singulier...,"[Formes d’adjectifs en ukrainien, ukrainien]",[form-of],[⁽ᵒᵖᵉʳᵗʸᵐ⁾],NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,опертими,uk,Ukrainien,adj,Forme d’adjectif,[{'glosses': ['Instrumental pluriel de опе́рти...,"[Formes d’adjectifs en ukrainien, ukrainien]",[form-of],[⁽ᵒᵖᵉʳᵗʸᵐʸ⁾],NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,опертих,uk,Ukrainien,adj,Forme d’adjectif,"[{'glosses': ['Génitif pluriel de опе́ртий.'],...","[Formes d’adjectifs en ukrainien, ukrainien]",[form-of],[⁽ᵒᵖᵉʳᵗʸᵏʰ⁾],NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,оперті,uk,Ukrainien,adj,Forme d’adjectif,[{'glosses': ['Nominatif pluriel de опе́ртий.'...,"[Formes d’adjectifs en ukrainien, ukrainien]",[form-of],[⁽ᵒᵖᵉʳᵗⁱ⁾],NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
999995,rabbriccicherete,it,Italien,verb,Forme de verbe,[{'glosses': ['Deuxième personne du pluriel du...,"[Formes de verbes en italien, italien]",[form-of],NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
999996,rabbruscherete,it,Italien,verb,Forme de verbe,[{'glosses': ['Deuxième personne du pluriel du...,"[Formes de verbes en italien, italien]",[form-of],NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
999997,rabescherete,it,Italien,verb,Forme de verbe,[{'glosses': ['Deuxième personne du pluriel du...,"[Formes de verbes en italien, italien]",[form-of],NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
999998,raccherete,it,Italien,verb,Forme de verbe,[{'glosses': ['Deuxième personne du pluriel du...,"[Formes de verbes en italien, italien]",[form-of],NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
